# High Low distribution by Time and RSI - Strategy - Backtesting 

### Import Library

In [1]:
import warnings
warnings.filterwarnings('ignore')
#
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy
# from backtesting._plotting import set_bokeh_output
# set_bokeh_output(notebook=False)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120

Loading BokehJS ...

### Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
data = dataset.copy()

In [4]:
data = data[data.index > '2020-11-01 00:00:00']
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
# data = data[data.index > '2024-11-01 00:00:00']

In [5]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2025-10-21 14:10:00,1927.5,1933.0,1918.7,1922.0,16658
2025-10-21 14:15:00,1922.0,1927.5,1919.2,1921.5,8361
2025-10-21 14:20:00,1921.5,1926.0,1902.9,1906.7,16009


In [6]:
data["max3"] = data["High"].rolling(3).max()
data["min3"] = data["Low"].rolling(3).min()
data["RSI_14"] = ta.rsi(data["Close"], length=14)

### Cal max High and min Low from 9h to current time

In [7]:
%%time
data['current_ismax'] = False
data['current_ismin'] = False
for i, row in data.iterrows():
    current_date = row.name.strftime('%Y-%m-%d ').format()
    current_time = row.name
    if row.name == 9:
        continue
    data_from_start_day = data[(data.index < current_time) & (data.index > current_date+' 08:59:00')]
    max_price = row['High']
    min_price = row['Low']
    for k, wrow in data_from_start_day.iterrows():
        if wrow['Low'] < min_price:
            min_price = wrow['Low']
        if wrow['High'] > max_price:
            max_price = wrow['High']
        #
    data.at[i, 'current_ismax'] = True if max_price == row['High'] else False
    data.at[i, 'current_ismin'] = True if min_price == row['Low'] else False

CPU times: user 1min 49s, sys: 18.3 ms, total: 1min 49s
Wall time: 1min 49s


In [8]:
data

,Open,High,Low,Close,Volume,max3,min3,RSI_14,current_ismax,current_ismin
Date,,,,,,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910,NaN,NaN,NaN,True,True
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670,NaN,NaN,0.000000,True,True
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329,900.2,898.7,1.086957,False,False
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722,900.2,898.2,0.983359,False,True
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939,900.0,896.5,0.940456,False,True
...,...,...,...,...,...,...,...,...,...,...
2025-10-21 14:10:00,1927.5,1933.0,1918.7,1922.0,16658,1933.0,1904.6,64.599617,True,False
2025-10-21 14:15:00,1922.0,1927.5,1919.2,1921.5,8361,1933.0,1917.2,64.187468,False,False
2025-10-21 14:20:00,1921.5,1926.0,1902.9,1906.7,16009,1933.0,1902.9,53.339461,False,False


In [ ]:
def cal_signal(r):
    signal = ''
    if r.name.hour == 10:
        if r['RSI'] < 40 and r['Close'] > r['Close_s1']:
        signal = 'long'
    elif r['Close'] > r['Close_s1']:
        signal = 'long'
    return signal

def prepare_data(df):
    df['signal'] = df.apply(lambda r: cal_signal(r), axis=1)
    return df

In [ ]:
prepared_data = prepare_data(data)
prepared_data.dropna(inplace=True)

In [ ]:
class MainStrategy(Strategy):
    reward_on_risk = 3
    def init(self):
        self._broker._cash = 1500
        super().init()

    def next(self):
        super().next()
        _time = self.data.index
        current_time = _time[-1]
        if current_time.hour == 14 and current_time.minute >= 25:
            if self.position.is_long or self.position.is_short:
                self.position.close()
            return

        if self.position:
            return 
        signal = self.data.signal[-1]
        close_price = self.data.Close[-1]
        open_price = self.data.Open[-1]
        if signal == 'long':
            buy_price = close_price
            sl = buy_price - 3
            tp = buy_price + 6
            self.buy(size=1, sl=sl, tp=tp)
            # self.buy(size=1, sl=sl, tp=tp, limit=buy_price)
        elif signal == 'short':
            sell_price = close_price
            sl = sell_price + 3
            tp = sell_price - 6
            self.sell(size=1, sl=sl, tp=tp)
            # self.sell(size=1, sl=sl, tp=tp, limit=sell_price)

In [ ]:
bt = Backtest(prepared_data, MainStrategy, commission=.0003, exclusive_orders=True)
stats = bt.run()

In [ ]:
stats

In [ ]:
stats['_trades']

In [ ]:
# bt.plot()

In [ ]:
copy_trades = stats['_trades'].copy()
copy_trades['cum_sum'] = copy_trades['PnL'].cumsum()
X = np.array(range(0, len(copy_trades['cum_sum'])))
Y = copy_trades['cum_sum']
# Plotting the Graph
plt.plot(X, Y)
plt.title("Curve plotted for returns")
plt.xlabel("X")
plt.ylabel("Rerurns")
plt.show()